<a href="https://colab.research.google.com/github/mnechromancer/JUMP-exercises/blob/main/PySpark_Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Apache Spark is written is Scala, an Object-Functional language that runs on the Java Virtual Machine(jvm).  In order to run Spark, we must install a compatiable version of Java into our virtual environment.  In this case, Java 8.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Next, we must install a version of Spark from an online repository, and unarchive the file.

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

PySpark is a Python API for Spark, allowing us to access Spark Functionality through Python code.  The findsaprk package will allow us to load Spark into our Python environment

In [ ]:
!pip install -q findspark

Setting Up Environment Variables

Python will use these environment variables to access the 

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

Setting up a Session

Spark Session is the entry point to SparkSQL, which will allow uas to create Spark RDDs and DataFrames

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

# Make df from csv

In [ ]:
cars = spark.read.options(header="true", inferSchema="true").csv('/content/car_sales_table.csv')
cars.show()

+-------+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+
|StockID|        Make|        Model|ColorID|VehicleType|CostPrice|SpareParts|LaborCost|Registration_Date|Mileage|PurchaseDate|
+-------+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+
|      1| Rolls Royce|     Camargue|      1|     Saloon|    50000|       500|      750|         1/8/1985|  52500|    1/1/2012|
|      2|Aston Martin|          DBS|      2|      Coupe|    75000|       600|      550|         5/9/2007|  75000|    1/1/2012|
|      3| Rolls Royce| Silver Ghost|      3|     Saloon|    75000|       750|      550|         9/8/2006|  52500|    1/1/2012|
|      4| Rolls Royce| Silver Ghost|      2|     Saloon|    88000|        85|      550|         1/5/1987| 127220|    1/1/2012|
|      5| Rolls Royce|     Camargue|      5|     Saloon|    62000|      2500|      550|         1/8/1985|  5250

# This won't work

In [ ]:
cars.index = cars.StockID
cars.show()

+-------+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+
|StockID|        Make|        Model|ColorID|VehicleType|CostPrice|SpareParts|LaborCost|Registration_Date|Mileage|PurchaseDate|
+-------+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+
|      1| Rolls Royce|     Camargue|      1|     Saloon|    50000|       500|      750|         1/8/1985|  52500|    1/1/2012|
|      2|Aston Martin|          DBS|      2|      Coupe|    75000|       600|      550|         5/9/2007|  75000|    1/1/2012|
|      3| Rolls Royce| Silver Ghost|      3|     Saloon|    75000|       750|      550|         9/8/2006|  52500|    1/1/2012|
|      4| Rolls Royce| Silver Ghost|      2|     Saloon|    88000|        85|      550|         1/5/1987| 127220|    1/1/2012|
|      5| Rolls Royce|     Camargue|      5|     Saloon|    62000|      2500|      550|         1/8/1985|  5250

# This kind of works

In [ ]:
cars = cars.withColumnRenamed("StockID", "Index")
cars.show()

+-----+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+
|Index|        Make|        Model|ColorID|VehicleType|CostPrice|SpareParts|LaborCost|Registration_Date|Mileage|PurchaseDate|
+-----+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+
|    1| Rolls Royce|     Camargue|      1|     Saloon|    50000|       500|      750|         1/8/1985|  52500|    1/1/2012|
|    2|Aston Martin|          DBS|      2|      Coupe|    75000|       600|      550|         5/9/2007|  75000|    1/1/2012|
|    3| Rolls Royce| Silver Ghost|      3|     Saloon|    75000|       750|      550|         9/8/2006|  52500|    1/1/2012|
|    4| Rolls Royce| Silver Ghost|      2|     Saloon|    88000|        85|      550|         1/5/1987| 127220|    1/1/2012|
|    5| Rolls Royce|     Camargue|      5|     Saloon|    62000|      2500|      550|         1/8/1985|  52500|    1/1/2012|


# Indexes (sort of)

In [ ]:
cars.collect()[2:5]

[Row(Index=3, Make='Rolls Royce', Model='Silver Ghost', ColorID=3, VehicleType='Saloon', CostPrice=75000, SpareParts=750, LaborCost=550, Registration_Date='9/8/2006', Mileage=52500, PurchaseDate='1/1/2012'),
 Row(Index=4, Make='Rolls Royce', Model='Silver Ghost', ColorID=2, VehicleType='Saloon', CostPrice=88000, SpareParts=85, LaborCost=550, Registration_Date='1/5/1987', Mileage=127220, PurchaseDate='1/1/2012'),
 Row(Index=5, Make='Rolls Royce', Model='Camargue', ColorID=5, VehicleType='Saloon', CostPrice=62000, SpareParts=2500, LaborCost=550, Registration_Date='1/8/1985', Mileage=52500, PurchaseDate='1/1/2012')]

In [ ]:
cars.collect()[3:6]

[Row(Index=4, Make='Rolls Royce', Model='Silver Ghost', ColorID=2, VehicleType='Saloon', CostPrice=88000, SpareParts=85, LaborCost=550, Registration_Date='1/5/1987', Mileage=127220, PurchaseDate='1/1/2012'),
 Row(Index=5, Make='Rolls Royce', Model='Camargue', ColorID=5, VehicleType='Saloon', CostPrice=62000, SpareParts=2500, LaborCost=550, Registration_Date='1/8/1985', Mileage=52500, PurchaseDate='1/1/2012'),
 Row(Index=6, Make='Rolls Royce', Model='Camargue', ColorID=8, VehicleType='Saloon', CostPrice=125000, SpareParts=4000, LaborCost=550, Registration_Date='5/6/1986', Mileage=28750, PurchaseDate='1/1/2012')]

# Aggregates

In [ ]:
max_mileage_row = max(cars.select("Mileage").collect())
print(f"Maximum mileage: {max_mileage_row.Mileage}")

Maximum mileage: 161000


In [ ]:
CostPrice_col_ints = [i.CostPrice for i in cars.select("CostPrice").collect()]
costPrice_sum = sum(CostPrice_col_ints)
print(f"Sum of Cost: {costPrice_sum}")

Sum of Cost: 20510095


# Makes and models

In [ ]:
makes_and_models = cars.groupBy("Make", "Model")
makes_and_models.count().show()

+------------+-------------+-----+
|        Make|        Model|count|
+------------+-------------+-----+
|Aston Martin|          DBS|    4|
|Aston Martin|          DB7|   16|
|Aston Martin|       Zagato|    2|
| Rolls Royce|       Wraith|    2|
| Rolls Royce|     Camargue|   39|
|     Triumph|          TR7|    4|
|     Bentley|  Continental|   52|
|     Bentley|      Turbo R|   10|
|         TVR|       Tuscan|   11|
|Aston Martin|          DB4|    8|
|     Bentley|        Azure|    8|
|Aston Martin|       Rapide|    4|
|Aston Martin|          DB9|   54|
|         MGB|           GT|   36|
| Rolls Royce| Silver Ghost|   12|
| Rolls Royce|      Phantom|    2|
| Rolls Royce|Silver Shadow|    4|
|     Bentley|       Arnage|    2|
|Aston Martin|      Vantage|   12|
|      Jaguar|           XK|   95|
+------------+-------------+-----+
only showing top 20 rows



# Cost evaluations

In [ ]:
make_costs = cars.groupBy("Make").sum("CostPrice")
make_costs.show()

+------------+--------------+
|        Make|sum(CostPrice)|
+------------+--------------+
|      Jaguar|       4899700|
|         TVR|        612500|
| Rolls Royce|       4116225|
|         MGB|        342000|
|     Triumph|        655500|
|     Bentley|       3179400|
|Aston Martin|       6704770|
+------------+--------------+



In [ ]:
make_costs = cars.groupBy("Make").max("CostPrice")
make_costs.show()

+------------+--------------+
|        Make|max(CostPrice)|
+------------+--------------+
|      Jaguar|         67000|
|         TVR|         42500|
| Rolls Royce|        130000|
|         MGB|         13500|
|     Triumph|         22000|
|     Bentley|         67000|
|Aston Martin|        160000|
+------------+--------------+



# Add profits column

In [ ]:
added_profits = cars.withColumn("Profit", cars.CostPrice - (cars.LaborCost + cars.SpareParts))
added_profits.show()

+-----+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+------+
|Index|        Make|        Model|ColorID|VehicleType|CostPrice|SpareParts|LaborCost|Registration_Date|Mileage|PurchaseDate|Profit|
+-----+------------+-------------+-------+-----------+---------+----------+---------+-----------------+-------+------------+------+
|    1| Rolls Royce|     Camargue|      1|     Saloon|    50000|       500|      750|         1/8/1985|  52500|    1/1/2012| 48750|
|    2|Aston Martin|          DBS|      2|      Coupe|    75000|       600|      550|         5/9/2007|  75000|    1/1/2012| 73850|
|    3| Rolls Royce| Silver Ghost|      3|     Saloon|    75000|       750|      550|         9/8/2006|  52500|    1/1/2012| 73700|
|    4| Rolls Royce| Silver Ghost|      2|     Saloon|    88000|        85|      550|         1/5/1987| 127220|    1/1/2012| 87365|
|    5| Rolls Royce|     Camargue|      5|     Saloon|    62000|      2500| 